In [1]:
# =========================
# CELL 1 — Imports & config
# =========================

import numpy as np
import pandas as pd
from datetime import datetime

# --- Edit if needed ---
CSV_PATH = r"C:\Users\Vex\Desktop\main\dataset\matchday\clean_matchday_stat.csv"
TEAM_A = "Bayer Leverkusen"
TEAM_B = "Bayern Munich"


In [2]:
# =========================
# CELL 2 — Helpers
# =========================

def _clean_num(v):
    """
    Convert strings like '54%', '1,234' to numeric.
    Returns NaN on failure; leaves numerics as-is.
    """
    if isinstance(v, str):
        v = v.replace("%", "").replace(",", "").strip()
    return pd.to_numeric(v, errors="coerce")

def _parse_match_dt(row):
    """
    Build a datetime from season 'YYYY/YYYY', Date_day 'DD.MM' (or 'DD MM'), and Date_hour 'HH:MM'.
    Bundesliga spans two years:
      - Jul–Dec -> use season start year
      - Jan–May -> use start year + 1
    Falls back to a safe date if parsing fails.
    """
    # Normalize "day.month" or "day month"
    day_month = str(row.get("Date_day", "")).replace("-", ".").strip()
    hhmm = (str(row.get("Date_hour", "00:00")).strip() or "00:00")

    # Split out day & month
    if "." in day_month:
        parts = day_month.split(".", 1)
    else:
        parts = [p for p in day_month.split() if p]
    if len(parts) >= 2:
        d_str, m_str = parts[0], parts[1]
    else:
        d_str, m_str = "1", "7"  # default to 1 July if missing

    # Coerce to ints with guards
    try:
        d = int(float(d_str)); m = int(float(m_str))
    except Exception:
        d, m = 1, 7

    # Season-year logic (e.g., "2023/2024")
    season = str(row.get("season_year", "2023/2024"))
    start_year = int(season.split("/")[0]) if "/" in season else 2023
    year = start_year if m >= 7 else start_year + 1

    # Compose datetime; fallback path clamps month/day to safe bounds
    try:
        return datetime.strptime(f"{year:04d}-{m:02d}-{d:02d} {hhmm}", "%Y-%m-%d %H:%M")
    except Exception:
        return datetime(year, max(1, min(12, m)), max(1, min(28, d)), 0, 0)


In [3]:
# =========================
# CELL 3 — Build tables & preview
# =========================

def build_matchday_table(df, team_name):
    """
    Create a per-matchday table for a given team:
      matchday, date, venue, opponent, score, result (W/D/L).
    """
    rows = []
    for _, row in df.iterrows():
        home, away = row["home_team"], row["away_team"]
        home_g, away_g = _clean_num(row["home_score"]), _clean_num(row["away_score"])
        md_dt = _parse_match_dt(row)

        # Determine perspective & goals for/against
        if home == team_name:
            venue, opp = "Home", away
            gf, ga = home_g, away_g
        elif away == team_name:
            venue, opp = "Away", home
            gf, ga = away_g, home_g
        else:
            continue  # skip unrelated fixtures

        # Score string + result symbol
        if pd.isna(gf) or pd.isna(ga):
            score, result = "", ""
        else:
            score = f"{int(gf)}–{int(ga)}"
            result = "W" if gf > ga else ("L" if gf < ga else "D")

        rows.append({
            "match_dt": md_dt,
            "venue": venue,
            "opponent": opp,
            "score": score,
            "result": result,
        })

    # Chronological order + matchday index + printable date
    t = pd.DataFrame(rows).sort_values("match_dt").reset_index(drop=True)
    t["matchday"] = np.arange(1, len(t) + 1)
    t["date"] = t["match_dt"].dt.strftime("%Y-%m-%d")
    return t[["matchday", "date", "venue", "opponent", "score", "result"]]

# Load CSV and build both tables
df = pd.read_csv(CSV_PATH)
lev_table = build_matchday_table(df, TEAM_A)
bay_table = build_matchday_table(df, TEAM_B)

# Peek first 10 rows for each team
display(lev_table.head(10))
display(bay_table.head(10))


,matchday,date,venue,opponent,score,result
0,1,2023-08-19,Home,RB Leipzig,3–2,W
1,2,2023-08-26,Away,B. Monchengladbach,3–0,W
2,3,2023-09-02,Home,Darmstadt,5–1,W
3,4,2023-09-15,Away,Bayern Munich,2–2,D
4,5,2023-09-24,Home,Heidenheim,4–1,W
5,6,2023-09-30,Away,Mainz,3–0,W
6,7,2023-11-04,Away,Hoffenheim,3–2,W
7,8,2023-11-12,Home,Union Berlin,4–0,W
8,9,2023-11-25,Away,Werder Bremen,3–0,W
9,10,2023-12-03,Home,Dortmund,1–1,D


,matchday,date,venue,opponent,score,result
0,1,2023-08-18,Away,Werder Bremen,4–0,W
1,2,2023-08-27,Home,Augsburg,3–1,W
2,3,2023-09-02,Away,B. Monchengladbach,2–1,W
3,4,2023-09-15,Home,Bayer Leverkusen,2–2,D
4,5,2023-09-23,Home,Bochum,7–0,W
5,6,2023-09-30,Away,RB Leipzig,2–2,D
6,7,2023-11-04,Away,Dortmund,4–0,W
7,8,2023-11-11,Home,Heidenheim,4–2,W
8,9,2023-11-24,Away,FC Koln,1–0,W
9,10,2023-12-09,Away,Eintracht Frankfurt,1–5,L
